In [18]:
import pandas as pd 
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import scipy.stats as stats
import matplotlib.colors as mcolors

In [19]:
from read_wandb import wandb_results
api = wandb_results(project_id=f"nlp_project", wandb_username="gefen_and_tomer")

BASE_METRIC = "accuracy_per_mean_user_and_bot"


In [20]:
def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
    df = api.get_sweeps_results(sweeps, metric=metric)

    config_cols = [c for c in df.columns if
                   "config_" in c and c != "config_wandb_run_id" and c != "config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
    print(HPT_cols)
    if drop_HPT:
        df = df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]

    # dropping all non-numeric columns that are not in the HPT_cols list
    drop_names = [c for c in df.columns if
                  (c not in HPT_cols) and (c not in [metric, "epoch"]) and (df[c].dtype not in [np.number])]
    df = df.drop(drop_names, axis=1)

    if len(HPT_cols) == 1:  # has only one column of config_seed
        # add a dummy column to group by
        df["dummy"] = 'dummy'
        HPT_cols.append("dummy")

    grouped = df.groupby([c for c in HPT_cols if c != "config_seed"], dropna=False)
    mean_df = grouped.mean()
    std_df = grouped.std()
    if epoch == "best":
        best_col = mean_df[
            [c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)

    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())

    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    if epoch == "best": df_cols['epoch'] = best_col.apply(
        lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))

    summary_df = pd.DataFrame(df_cols, index=best_col.index)
    for d in drop_list:
        if d in summary_df.index:
            summary_df = summary_df.drop(d)
    if len(summary_df.index.names) == 1:
        return summary_df.rename_axis(group_name)
    else:
        return summary_df


def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound

# For HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [22]:
sweep_results = result_metric(["t6eff5sl"], "LLMs", drop_HPT=False, epoch="best")
sweep_results

Total number of sweeps: 1
Download sweep_id='t6eff5sl' data...


100%|██████████| 9/9 [00:00<00:00, 27021.29it/s]

['config_seed', 'config_basic_nature']



/var/folders/2z/8_s1frf91ml1xyr6_025bgk00000gn/T/ipykernel_7625/2749268688.py:14: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  (c not in HPT_cols) and (c not in [metric, "epoch"]) and (df[c].dtype not in [np.number])]


,mean,std,values,epoch,CI
LLMs,,,,,
18,0.830135,0.001448,"[0.8315151739333206, 0.8302621539443967, 0.828...",6,"(0.8286277597003583, 0.8315151739333206)"
19,0.832460,0.001709,"[0.8322588380631463, 0.8308611801313589, 0.834...",4,"(0.8308611801313589, 0.8342610048571851)"
20,0.829555,0.001522,"[0.8298332806344103, 0.8279128628691088, 0.830...",23,"(0.8279128628691087, 0.8309191196473545)"


# Result for a specific epoch

In [ ]:
# sweep_results = result_metric(["xewf30hs"], "LLMs", drop_HPT=False, epoch=10)
# sweep_results

In [ ]:
# def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
#     df = api.get_sweeps_results(sweeps, metric=metric) 
# 
#     config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
#     HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
#     print(HPT_cols)
#     if drop_HPT:
#         df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
#         HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
#         
#     grouped = df.groupby([c for c in HPT_cols if c != "config_seed"])
#     mean_df = grouped.mean()
#     std_df = grouped.std()
#     if epoch=="best":
#         best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
#     else:
#         best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
#     
#     result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
#     means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
#     stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())
# 
# 
#     df_cols = {'mean': means, 'std': stds, 'values': result.values}
#     if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")
# 
#     df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))
# 
#     summary_df = pd.DataFrame(df_cols, index=best_col.index)
#     for d in drop_list:
#         if d in summary_df.index:
#             summary_df=summary_df.drop(d)
#     if len(summary_df.index.names) == 1:
#         return summary_df.rename_axis(group_name)
#     else:
#         return summary_df
# 
# def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
#     bootstrapped_means = []
#     for _ in range(n_bootstrap):
#         sample = np.random.choice(data, size=len(data), replace=True)
#         bootstrapped_means.append(np.mean(sample))
#     lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
#     upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
#     return lower_bound, upper_bound
